Background: https://en.wikipedia.org/wiki/Satisfiability_modulo_theories

Library: https://github.com/Z3Prover/z3#z3

## Set-up, only needed initially

In [1]:
open System.Net

let wc = new WebClient()
//wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-ubuntu-16.04.zip", @"z3-4.6.0-x64-ubuntu-16.04.zip")
wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-win.zip", @"z3-4.6.0-x64-win.zip")
//macOS version too


In [2]:
//This will take a while
#r "System.IO.Compression.FileSystem.dll"

open System;
open System.IO;
open System.IO.Compression;

//ZipFile.ExtractToDirectory("z3-4.6.0-x64-ubuntu-16.04.zip", ".")
ZipFile.ExtractToDirectory("z3-4.6.0-x64-win.zip", ".")


Expression evaluation failed: The file 'C:\dev\OpenFS2018.4\notebooks\z3-4.6.0-x64-win\LICENSE.txt' already exists.
IOExceptionThe file 'C:\dev\OpenFS2018.4\notebooks\z3-4.6.0-x64-win\LICENSE.txt' already exists.
   at System.IO.__Error.WinIOError(Int32 errorCode, String maybeFullPath)
   at System.IO.FileStream.Init(String path, FileMode mode, FileAccess access, Int32 rights, Boolean useRights, FileShare share, Int32 bufferSize, FileOptions options, SECURITY_ATTRIBUTES secAttrs, String msgPath, Boolean bFromProxy, Boolean useLongPath, Boolean checkHost)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share)
   at System.IO.Compression.ZipFileExtensions.ExtractToFile(ZipArchiveEntry source, String destinationFileName, Boolean overwrite)
   at System.IO.Compression.ZipFileExtensions.ExtractToDirectory(ZipArchive source, String destinationDirectoryName)
   at System.IO.Compression.ZipFile.ExtractToDirectory(String sourceArchiveFileName, String de

## Reference and use Z3

In [3]:
#r "z3-4.6.0-x64-win/bin/Microsoft.Z3.dll"
//#r "z3-4.6.0-x64-ubuntu-16.04/bin/Microsoft.Z3.dll"

In [4]:
open Microsoft.Z3
let ctx = new Context()
ctx


Microsoft.Z3.Context

In [5]:
let petPurchasing() =
    use context = new Microsoft.Z3.Context()

    let dogCount = context.MkIntConst("dog")
    let catCount = context.MkIntConst("cat")
    let mouseCount = context.MkIntConst("mouse")

    //Expression for cost for the animals
    let dogsCost = context.MkMul(context.MkInt(1500), dogCount)
    let catsCost = context.MkMul(context.MkInt(100), catCount)
    let miceCost = context.MkMul(context.MkInt(25), mouseCount)

    let solver = context.MkSolver()
    //Ensure >= number of animals
    solver.Assert(context.MkGe(dogCount,context.MkInt(1)))
    solver.Assert(context.MkGe(catCount,context.MkInt(1)))
    solver.Assert(context.MkGe(mouseCount,context.MkInt(1)))
    
    let summedCost = context.MkAdd(dogsCost, catsCost, miceCost)

    //Ensure we spend exactly all our money
    solver.Assert(context.MkEq(context.MkInt(10000), summedCost))

    let status = solver.Check()

    match status with
    | Status.SATISFIABLE ->
            printfn "dogs: %s, cats: %s, mice: %s"
                (solver.Model.Eval(dogCount, true).ToString())
                (solver.Model.Eval(catCount, true).ToString())
                (solver.Model.Eval(mouseCount, true).ToString())
    | _ -> printfn "UNSAT" 

In [6]:
petPurchasing()

dogs: 6, cats: 9, mice: 4


Nicer DSL approach: https://github.com/dungpa/Z3Fs#examples

https://notebooks.azure.com/cgravill/libraries/z3sample/html/z3sample.ipynb

https://notebooks.azure.com/ian-buckley/libraries/probabilistic-programming